# Se importan las librerías

In [1]:
import os
import pandas as pd
import numpy as np
import pickle
import joblib

import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso, ElasticNet

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import (RandomForestRegressor, AdaBoostRegressor,
                             GradientBoostingRegressor, ExtraTreesRegressor)

from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                    StratifiedKFold)

from sklearn.decomposition import PCA

ruta_actual = os.getcwd()
## Funciones
os.chdir('../')  # Establecemos el directorio de trabajo
from utils import funciones as fn
from utils import listas as ls
os.chdir(ruta_actual)

In [2]:
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

# Se carga el csv con los datos preparados y escalados para el modelo

In [3]:
ruta_x_train_escalado = f'../data/clean/X_escalado.csv'
ruta_y_train = f'../data/clean/y.csv'
X_escalado = pd.read_csv(ruta_x_train_escalado,index_col=0)
y = pd.read_csv(ruta_y_train,index_col=0)

In [4]:
X_escalado

,numeric__Anyo,numeric__Kilometros,numeric__Potencia,text__Combustible_Diésel,text__Combustible_Eléctrico,text__Combustible_Gas licuado (GLP),text__Combustible_Gas natural (CNG),text__Combustible_Gasolina,text__Combustible_Híbrido,text__Combustible_Híbrido enchufable,...,text__Cluster_Modelo_compacto_pequeno,text__Cluster_Modelo_coupe_grande,text__Cluster_Modelo_coupe_pequeno,text__Cluster_Modelo_furgon_pasajero,text__Cluster_Modelo_furgon_trabajo,text__Cluster_Modelo_otro,text__Cluster_Modelo_sedan_grande,text__Cluster_Modelo_sedan_pequeno,text__Cluster_Modelo_suv_grande,text__Cluster_Modelo_suv_pequeno
0,0.559955,-0.240022,-0.350811,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.654176,0.094079,-1.071417,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.876260,-0.981926,2.115881,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.230806,-0.916977,1.533853,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.718107,-0.773070,1.811009,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35664,-0.863415,0.869765,2.836488,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
35665,1.034412,-1.112971,-0.239948,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
35666,-3.077546,0.873447,-2.180043,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35667,0.718107,-0.930523,-0.378526,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
y

,Precio
0,14490
1,1800
2,22800
3,25900
4,29418
...,...
35664,10990
35665,26000
35666,350
35667,13500


# Transformaciones sobre el csv Test (el modelo no va a entranar con estos datos, solo se utilizarán para evaluar el modelo)

### Se carga el pipeline con las transformaciones que se hicieron sobre el Train

In [6]:
pipeline = pickle.load(open('../model/pipeline_train.pkl','rb'))

### Se cargan los datos de Test

In [7]:
Test = pd.read_csv('../data/raw/Test.csv',index_col=0)

In [8]:
Test

,Marca,Modelo,Precio,Combustible,Anyo,Kilometros,Potencia,Vendedor_profesional,dealer,Comunidad_Autonoma
0,VOLKSWAGEN,Scirocco,8900,Gasolina,2009.0,180000,150.921053,False,7f0cb15c7942a445a5d62e929fab371d,Comunidad_de_Madrid
1,BMW,Serie 1,29990,Gasolina,2020.0,5000,136.000000,True,Luis Cars,Castilla_y_León
2,NISSAN,NV300,18700,Diésel,2017.0,51000,125.000000,True,Carsol,Andalucía
3,CITROEN,C3 Aircross,17400,Diésel,2018.0,17838,120.000000,True,Rondamovil,Andalucía
4,FORD,Mondeo,3500,Diésel,2007.0,279000,140.355769,False,e80a0702d314d055d05af996fe60cff9,Comunidad_Valenciana
...,...,...,...,...,...,...,...,...,...,...
9943,KIA,Carens,13490,Diésel,2017.0,29999,115.000000,True,Flexicar Sevilla,Andalucía
9944,OPEL,Corsa,1490,Gasolina,1999.0,120000,65.000000,True,Munain Motors,País_Vasco
9945,PEUGEOT,207,4900,Gasolina,2009.0,50200,75.000000,True,M&S,Comunidad_de_Madrid
9946,BMW,Serie 5,18950,Diésel,2016.0,96000,190.000000,True,ALD Sevilla,Andalucía


### Se separa la variable target del resto de datos del Test

In [9]:
X_test = Test.drop(columns=['Precio'])
y_test = Test['Precio']

In [10]:
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

### Se aplica la función que crea los Cluster para las variables `Marca` y `Modelo` y se eliminan las columnas no necesarias

In [11]:
fn.add_delete_columns(X_test, 'Marca',fn.cluster_marcas,'Modelo',fn.cluster_modelo,'dealer')

,Combustible,Anyo,Kilometros,Potencia,Vendedor_profesional,Comunidad_Autonoma,Cluster_Marca,Cluster_Modelo
0,Gasolina,2009.0,180000,150.921053,False,Comunidad_de_Madrid,Mid_Premium,coupe_pequeno
1,Gasolina,2020.0,5000,136.000000,True,Castilla_y_León,Premium,compacto_pequeno
2,Diésel,2017.0,51000,125.000000,True,Andalucía,Standard,otro
3,Diésel,2018.0,17838,120.000000,True,Andalucía,Standard,compacto_pequeno
4,Diésel,2007.0,279000,140.355769,False,Comunidad_Valenciana,Standard,sedan_grande
...,...,...,...,...,...,...,...,...
9943,Diésel,2017.0,29999,115.000000,True,Andalucía,Standard,suv_grande
9944,Gasolina,1999.0,120000,65.000000,True,País_Vasco,Low_Cost,compacto_pequeno
9945,Gasolina,2009.0,50200,75.000000,True,Comunidad_de_Madrid,Standard,compacto_pequeno
9946,Diésel,2016.0,96000,190.000000,True,Andalucía,Premium,sedan_grande


### Se aplica el pipeline con las transofrmaciones

In [12]:
X_test_trans= pipeline.transform(X_test).toarray()
X_test_escalado = pd.DataFrame(
    X_test_trans, columns=pipeline.get_feature_names_out())
X_test_escalado

,numeric__Anyo,numeric__Kilometros,numeric__Potencia,text__Combustible_Diésel,text__Combustible_Eléctrico,text__Combustible_Gas licuado (GLP),text__Combustible_Gas natural (CNG),text__Combustible_Gasolina,text__Combustible_Híbrido,text__Combustible_Híbrido enchufable,...,text__Cluster_Modelo_compacto_pequeno,text__Cluster_Modelo_coupe_grande,text__Cluster_Modelo_coupe_pequeno,text__Cluster_Modelo_furgon_pasajero,text__Cluster_Modelo_furgon_trabajo,text__Cluster_Modelo_otro,text__Cluster_Modelo_sedan_grande,text__Cluster_Modelo_sedan_pequeno,text__Cluster_Modelo_suv_grande,text__Cluster_Modelo_suv_pequeno
0,-0.547110,0.765569,0.617049,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.192564,-1.260479,0.203502,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.718107,-0.727918,-0.101370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.876260,-1.111848,-0.239948,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.863415,1.911733,0.324225,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9943,0.718107,-0.971055,-0.378526,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9944,-2.128632,0.070924,-1.764308,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9945,-0.547110,-0.737180,-1.487152,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9946,0.559955,-0.206934,1.700147,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Parametros de GridSearchCV

### se crea un diccionario con los modelos sobbre los que se aplicarán los hiperparametros

In [13]:
mejores_modelos = {
    'RandomForest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'AdaBoost': AdaBoostRegressor(DecisionTreeRegressor()),
    'ExtraTree': ExtraTreesRegressor()
} # diccionario para poder iterar y realizar las 3 evaluaciones de una vez

In [14]:
grid_RandomForest = {
    'n_estimators':[200,400],
    'max_features' : ['auto', 'sqrt'],
    'max_depth': [None, 90],
    'min_samples_leaf' : [2, 4, 6],
    'min_samples_split' : [2, 5, 10]
} # parametros para randomforest

In [15]:
grid_XGBoost = {
    'n_estimators': [200, 400],
    'max_depth': [None, 90],
    'eta': [0.05, 0.1, 0.01],
    'gamma': [0, 2, 4],
} # parametros para XGBoost

In [16]:
grid_AdaBoost = {
    'n_estimators': [200, 400, 600],
    'learning_rate': [0.05, 0.1, 0.20],
    'loss' :['linear', 'square']
} # parametros para Adaboost

In [17]:
grid_ExtraTree = {
    'n_estimators':[200,400],
    'max_features' : ['sqrt', 'log2', None],
    'max_depth': [None,90],
    'min_samples_leaf' : [2, 4, 6],
    'min_samples_split' : [2, 5, 10]
} # parametros para randomforest

### Se crea un diccionario con todas las grid de los distintos modelos

In [18]:
grid = {
    'RandomForest': grid_RandomForest,
    'XGBoost': grid_XGBoost,
    'AdaBoost': grid_AdaBoost,
    'ExtraTree': grid_ExtraTree
} # diccionario con las grid de los distintos modelos para poder iterar

# Optimización de los modelos

### Se crea un diccionario para almacenar los modelos entrenados

In [19]:
modelos_entrenados = {} # diccionario vacio para almacenar los modelos entrenados

Por cada uno de los modelos, se realizará la busqueda de los mejores parametros con GridSearchVC y se evaluará el modelo resultante sobre los datos del Test, de esta manera tendremos las métricas de Train con los modelos optimizados y las métricas sobre los datos de Test (el modleo no ha visto los datos Test)

In [20]:
tabla_metricas = pd.DataFrame()

for k,v in mejores_modelos.items(): # se itera sobre los 3 mejores modelos selleccionados anteriormente
    print(k)
    red_parametros = grid[k]
    gridsearch = GridSearchCV(estimator=v,param_grid=red_parametros,n_jobs=-1,scoring='neg_mean_squared_error') # gridsearchcv para encontrar los mejores parametros
    modelos_entrenados[k]=gridsearch.fit(X_escalado, y) # entrenamiento del modelo con todos los datos del train
    y_pred = modelos_entrenados[k].predict(X_test_escalado) # prediccion para cada comunidad autónoma
    tabla_metricas = tabla_metricas.append({
        'MAE' : mean_absolute_error(y_test,y_pred),
        'MSE' : mean_squared_error(y_test,y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test,y_pred)),
        'MAPE': mean_absolute_percentage_error(y_test,y_pred),
        'MODELO': k
        },ignore_index=True) # tabla con el resultado de cada metrica para cada modelo y comunidad autónoma

RandomForest


c:\Users\saula\miniconda3\envs\the_bridge_22\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\saula\miniconda3\envs\the_bridge_22\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


XGBoost
AdaBoost
ExtraTree


c:\Users\saula\miniconda3\envs\the_bridge_22\lib\site-packages\sklearn\model_selection\_search.py:910: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


### Se muestran las métricas del Train

In [21]:
for k in modelos_entrenados.keys():
    print(f'MSE para modelo {k}: {modelos_entrenados[k].best_score_}')

MSE para modelo RandomForest: -5787550.165813404
MSE para modelo XGBoost: -5230687.566568364
MSE para modelo AdaBoost: -5565802.177831943
MSE para modelo ExtraTree: -5487976.296855401


### Se muestran las métricas del Test

In [22]:
tabla_metricas.sort_values(by=['MSE'],ascending=True)

,MAE,MSE,RMSE,MAPE,MODELO
1,2969.246626,9.730367e+07,9864.262022,0.226695,XGBoost
2,3126.277751,1.075594e+08,10371.084676,0.255775,AdaBoost
0,3157.856174,1.086237e+08,10422.267298,0.235574,RandomForest
3,3157.985311,1.104554e+08,10509.777956,0.234308,ExtraTree


Después de analizar los resultados de las métricas el modelo elgido será el `XGBoost`

### Se guardan los modelos en formato pkl

In [23]:
pickle.dump(modelos_entrenados['RandomForest'], open('../model/modelo_RF.pkl', 'wb'))
pickle.dump(modelos_entrenados['XGBoost'], open('../model/modelo_XGB.pkl', 'wb'))
pickle.dump(modelos_entrenados['AdaBoost'], open('../model/modelo_ADA.pkl', 'wb'))
pickle.dump(modelos_entrenados['ExtraTree'], open('../model/modelo_ETR.pkl', 'wb'))

# KMeans

Para la evaluación de los modelos con KMeans se realizará el mismo proceso anterior, añadiendo la predicción de los cluster sobre los datos de Test. Se optimizarán los modelos por cada uno de los cluster como ya se hizo en la evaluzación inicial de los modelos

### Se carga el KMeans guardado en la evaluación del modelo y se aplica sobre los datos Test ya transformados

In [24]:
knn_2 = pickle.load(open('../model/knn_2.pkl','rb'))
cluster = knn_2.predict(X_test_escalado)
X_test_km = X_test_escalado.copy()
X_test_km["Cluster"]=cluster

In [25]:
Test_km = X_test_km.copy()
Test_km['Precio']=y_test

In [26]:
Test_km

,numeric__Anyo,numeric__Kilometros,numeric__Potencia,text__Combustible_Diésel,text__Combustible_Eléctrico,text__Combustible_Gas licuado (GLP),text__Combustible_Gas natural (CNG),text__Combustible_Gasolina,text__Combustible_Híbrido,text__Combustible_Híbrido enchufable,...,text__Cluster_Modelo_coupe_pequeno,text__Cluster_Modelo_furgon_pasajero,text__Cluster_Modelo_furgon_trabajo,text__Cluster_Modelo_otro,text__Cluster_Modelo_sedan_grande,text__Cluster_Modelo_sedan_pequeno,text__Cluster_Modelo_suv_grande,text__Cluster_Modelo_suv_pequeno,Cluster,Precio
0,-0.547110,0.765569,0.617049,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,8900
1,1.192564,-1.260479,0.203502,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,29990
2,0.718107,-0.727918,-0.101370,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,18700
3,0.876260,-1.111848,-0.239948,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,17400
4,-0.863415,1.911733,0.324225,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,3500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9943,0.718107,-0.971055,-0.378526,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,13490
9944,-2.128632,0.070924,-1.764308,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1490
9945,-0.547110,-0.737180,-1.487152,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,4900
9946,0.559955,-0.206934,1.700147,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,18950


### Se cargan los datos Train, en este caso no se separa la variable target ya que se necesita para poder separar los datos en los cluster

In [27]:
Train_km = pd.read_csv('../data/clean/Train_km.csv',index_col=0)

In [28]:
modelos_entrenados_cluster = {} # diccionario vacio para almacenar los modelos entrenados

### Se aplica GridSerachCV de la misma manera que en el caso anterior pero en este caso se evaluarán los modelos por cada uno de los 2 cluster

In [29]:
tabla_metricas_cluster = pd.DataFrame()

for clu in Test_km['Cluster'].unique():
    for k,v in mejores_modelos.items(): # se itera sobre los 3 mejores modelos selleccionados anteriormente
        print(k)
        red_parametros = grid[k]
        gridsearch = GridSearchCV(estimator=v,param_grid=red_parametros,n_jobs=-1,scoring='neg_mean_squared_error') # gridsearchcv para encontrar los mejores parametros
        modelos_entrenados_cluster[f'{k}_{clu}']=gridsearch.fit(
            Train_km[Train_km.Cluster == clu].drop(
                columns=['Precio','Cluster']),
            Train_km[Train_km.Cluster == clu]['Precio'] ) # entrenamiento del modelo con todos los datos del train
        y_pred = modelos_entrenados_cluster[f'{k}_{clu}'].predict(
            Test_km[Test_km.Cluster == clu].drop(columns=['Precio','Cluster'])) # prediccion para cada comunidad autónoma
        tabla_metricas_cluster = tabla_metricas_cluster.append({
            'MAE' : mean_absolute_error(
                Test_km[Test_km.Cluster == clu]['Precio'],y_pred),
            'MSE' : mean_squared_error(
                Test_km[Test_km.Cluster == clu]['Precio'],y_pred),
            'RMSE': np.sqrt(mean_squared_error(
                Test_km[Test_km.Cluster == clu]['Precio'],y_pred)),
            'MAPE': mean_absolute_percentage_error(
                Test_km[Test_km.Cluster == clu]['Precio'],y_pred),
            'MODELO': k,
            'CLUSTER':clu
            },ignore_index=True) # tabla con el resultado de cada metrica para cada modelo y comunidad autónoma

RandomForest


c:\Users\saula\miniconda3\envs\the_bridge_22\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


XGBoost
AdaBoost
ExtraTree
RandomForest


c:\Users\saula\miniconda3\envs\the_bridge_22\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


XGBoost
AdaBoost
ExtraTree


In [30]:
for k in modelos_entrenados_cluster.keys():
    print(f'MSE para modelo {k}: {modelos_entrenados_cluster[k].best_score_}')

MSE para modelo RandomForest_1: -4803229.332193328
MSE para modelo XGBoost_1: -4219362.239052712
MSE para modelo AdaBoost_1: -4626969.449644882
MSE para modelo ExtraTree_1: -4681963.903630562
MSE para modelo RandomForest_0: -6384815.824425523
MSE para modelo XGBoost_0: -5958741.612113949
MSE para modelo AdaBoost_0: -6256641.006222598
MSE para modelo ExtraTree_0: -6027753.05755993


In [31]:
tabla_metricas_cluster.sort_values(by=['CLUSTER','MSE'],ascending=True)

,MAE,MSE,RMSE,MAPE,MODELO,CLUSTER
5,3821.645576,1.525718e+08,12351.996204,0.131152,XGBoost,0
6,3965.180251,1.621663e+08,12734.452270,0.139472,AdaBoost,0
4,4007.195247,1.642273e+08,12815.120391,0.134951,RandomForest,0
7,3978.375180,1.654637e+08,12863.270264,0.132684,ExtraTree,0
1,1753.289187,2.109700e+07,4593.147582,0.378172,XGBoost,1
2,1786.546810,2.281393e+07,4776.392558,0.374042,AdaBoost,1
0,1826.962435,2.286817e+07,4782.067820,0.390250,RandomForest,1
3,1876.631287,2.472977e+07,4972.903951,0.392016,ExtraTree,1


En los resultados se puede ver que uno de los cluster tiene mejor métrica que la obtenida sin KMeans, pero el otro cluster tiene peor resultado, por esta razón se elige como modelo final el XGBoost del modelo entrenado sin KMeans